<a href="https://colab.research.google.com/github/mystakhs/LLM_notebooks/blob/main/ch06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
これは <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>（<a href="https://sebastianraschka.com">Sebastian Raschka</a>著）の補足コードである<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第6章: テキスト分類のファインチューニング

In [1]:
!git clone https://github.com/rasbt/LLMs-from-scratch.git
!cd LLMs-from-scratch
!pip install -e ./LLMs-from-scratch

Cloning into 'LLMs-from-scratch'...
remote: Enumerating objects: 5514, done.
remote: Counting objects: 100% (1703/1703), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 5514 (delta 1500), reused 1316 (delta 1316), pack-reused 3811 (from 3)
Receiving objects: 100% (5514/5514), 13.07 MiB | 11.06 MiB/s, done.
Resolving deltas: 100% (3478/3478), done.
Obtaining file:///content/LLMs-from-scratch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.4 

In [2]:
!mv LLMs-from-scratch LLMs_from_scratch

In [4]:
from importlib.metadata import version

pkgs = ["matplotlib",  # Plotting library
        "numpy",       # PyTorch & TensorFlow dependency
        "tiktoken",    # Tokenizer
        "torch",       # Deep learning library
        "tensorflow",  # For OpenAI's pretrained weights
        "pandas"       # Dataset loading
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.10.0
numpy version: 2.0.2
tiktoken version: 0.9.0
torch version: 2.6.0+cu124
tensorflow version: 2.18.0
pandas version: 2.2.2


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/chapter-overview.webp" width=500px>

## 6.1 ファインチューニングの異なるカテゴリ

- このセクションにはコードが含まれていない

- 言語モデルをファインチューニングする最も一般的な方法は、指示ファインチューニングと分類ファインチューニングである
- 下図に示すインストラクションファインチューニングは次章のトピックである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/instructions.webp" width=500px>

- この章のトピックである分類ファインチューニングは、もし機械学習の背景があればすでに馴染みがある手順である（たとえば、手書き数字を分類するために畳み込みネットワークを訓練するなど）。
- 分類ファインチューニングでは、特定のクラスラベル（例:「spam」「not spam」）の数があり、モデルはそれらを出力できる
- 分類ファインチューニングされたモデルは、学習時に見たクラス（たとえば「spam」「not spam」）のみを予測できるのに対し、インストラクションファインチューニングされたモデルは通常、多岐にわたるタスクを実行できる
- 分類ファインチューニングされたモデルは非常に特化したモデルと考えられる。多くのタスクをこなす汎用モデルを作るより、特化したモデルを作るほうがはるかに容易である

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/spam-non-spam.webp" width=500px>

## 6.2 データセットの準備

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-1.webp" width=500px>

- このセクションでは、分類ファインチューニングに使用するデータセットを準備する
- 「spam」と「non-spam」のテキストメッセージを含むデータセットを使用して、モデルをそれらを分類可能にする
- まず、データセットをダウンロードして解凍する

In [ ]:
import urllib.request
import zipfile
import os
from pathlib import Path

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return

    # Downloading the file
    with urllib.request.urlopen(url) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())

    # Unzipping the file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    # Add .tsv file extension
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as {data_file_path}")

try:
    download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)
except (urllib.error.HTTPError, urllib.error.URLError, TimeoutError) as e:
    print(f"Primary URL failed: {e}. Trying backup URL...")
    url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/sms%2Bspam%2Bcollection.zip"
    download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

File downloaded and saved as sms_spam_collection/SMSSpamCollection.tsv


- このデータセットはタブ区切りのテキストファイルとして保存されており、pandasのDataFrameに読み込むことができる

In [ ]:
import pandas as pd

df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


- クラス分布を確認すると、「ham」（非スパム）のほうが「spam」よりもはるかに多いことがわかる

In [ ]:
print(df["Label"].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


- 学習の便宜上、そして教育用の目的として小さなデータセットを使うほうがよいので、データセットをサンプリングして各クラスを747件ずつに揃える
- （不均衡なクラス分布を扱う方法は他にも多数存在するが、本書の焦点はLLMであるため割愛する。詳しくは [`imbalanced-learn` ユーザーガイド](https://imbalanced-learn.org/stable/user_guide.html)を参照）

In [ ]:
def create_balanced_dataset(df):

    # Count the instances of "spam"
    num_spam = df[df["Label"] == "spam"].shape[0]

    # Randomly sample "ham" instances to match the number of "spam" instances
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)

    # Combine ham "subset" with "spam"
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])

    return balanced_df


balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

Label
ham     747
spam    747
Name: count, dtype: int64


- 次に、文字列であるクラスラベル「ham」「spam」を整数ラベル0と1に変換する:

In [ ]:
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

In [ ]:
balanced_df

,Label,Text
4307,0,Awww dat is sweet! We can think of something t...
4138,0,Just got to &lt;#&gt;
4831,0,"The word ""Checkmate"" in chess comes from the P..."
4461,0,This is wishing you a great day. Moji told me ...
5440,0,Thank you. do you generally date the brothas?
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


- 次に、データフレームをランダムに分割し、訓練、バリデーション、テストの各サブセットを作成する関数を定義する

In [ ]:
def random_split(df, train_frac, validation_frac):
    # Shuffle the entire DataFrame
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # Calculate split indices
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # Split the DataFrame
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)
# Test size is implied to be 0.2 as the remainder

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

## 6.3 データローダーの作成

- テキストメッセージは長さが異なる。バッチで複数のサンプルをまとめる場合、以下の2つの方法のいずれかを取れる:
  1. データセットやバッチ内の最短メッセージにすべて切り詰める
  2. データセットやバッチ内の最長メッセージに合わせてパディングする

- ここでは2を選択し、データセット内の最長メッセージに合わせてパディングを行う
- パディングには `<|endoftext|>` トークンを使う（第2章で述べたとおり）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/pad-input-sequences.webp?123" width=500px>

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


- 以下の `SpamDataset` クラスは、訓練データセット内の最長シーケンスを特定し、それに合わせてパディングトークンを追加してシーケンス長を揃える

In [ ]:
import torch
from torch.utils.data import Dataset

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            # Truncate sequences if they are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # Pad sequences to the longest sequence
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length
        # Note: A more pythonic version to implement this method
        # is the following, which is also used in the next chapter:
        # return max(len(encoded_text) for encoded_text in self.encoded_texts)

In [ ]:
train_dataset = SpamDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

print(train_dataset.max_length)

120


- バリデーションセットとテストセットも訓練データの最長シーケンス長に合わせてパディングを行う
- 訓練セット内の最長シーケンスより長いバリデーション／テストのサンプルは、`encoded_text[:self.max_length]` によって切り詰められる
- この挙動は必須ではなく、もし望むならバリデーションセットとテストセットでも `max_length=None` を指定してそれぞれの最長を使うことも可能である

In [ ]:
val_dataset = SpamDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
test_dataset = SpamDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

- 次に、このデータセットを使ってデータローダーを生成する。これは以前の章でのデータローダー作成方法と同様である

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/batch.webp" width=500px>

In [ ]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

- 検証のため、データローダーをイテレートして、バッチ内に8つの訓練サンプルが含まれ、それぞれが120トークンであることを確認する

In [ ]:
print("Train loader:")
for input_batch, target_batch in train_loader:
    pass

print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

Train loader:
Input batch dimensions: torch.Size([8, 120])
Label batch dimensions torch.Size([8])


- 最後に、各データセットに含まれるバッチ数を出力する

In [ ]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

130 training batches
19 validation batches
38 test batches


## 6.4 事前学習済みの重みを用いたモデルの初期化

- このセクションでは、前章で扱った事前学習済みモデルを初期化する

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-2.webp" width=500px>

In [3]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

assert train_dataset.max_length <= BASE_CONFIG["context_length"], (
    f"Dataset length {train_dataset.max_length} exceeds model's context "
    f"length {BASE_CONFIG['context_length']}. Reinitialize data sets with "
    f"`max_length={BASE_CONFIG['context_length']}`"
)

NameError: name 'train_dataset' is not defined

In [ ]:
from gpt_download import download_and_load_gpt2
# from previous_chapters import GPTModel, load_weights_into_gpt
from LLMs_from_scratch.pkg.llms_from_scratch.ch04 import GPTModel
from LLMs_from_scratch.pkg.llms_from_scratch.ch05 import download_and_load_gpt2, load_weights_into_gpt
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel
# from llms_from_scratch.ch05 import download_and_load_gpt2, load_weights_into_gpt

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")

model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

File already exists and is up-to-date: gpt2/124M/checkpoint
File already exists and is up-to-date: gpt2/124M/encoder.json
File already exists and is up-to-date: gpt2/124M/hparams.json
File already exists and is up-to-date: gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/124M/model.ckpt.index
File already exists and is up-to-date: gpt2/124M/model.ckpt.meta
File already exists and is up-to-date: gpt2/124M/vocab.bpe


- モデルが正しく読み込まれたことを確認するため、テキスト生成が正しく行われるかをチェックする

In [ ]:
# from previous_chapters import (
#     generate_text_simple,
#     text_to_token_ids,
#     token_ids_to_text
# )
from LLMs_from_scratch.pkg.llms_from_scratch.ch05 import (
    generate_text_simple,
    text_to_token_ids,
    token_ids_to_text
)

# Alternatively:
# from llms_from_scratch.ch05 import (
#    generate_text_simple,
#    text_to_token_ids,
#    token_ids_to_text
# )

text_1 = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_1, tokenizer),
    max_new_tokens=15,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

Every effort moves you forward.

The first step is to understand the importance of your work


- モデルを分類器としてファインチューニングする前に、プロンプトだけで既にスパムを分類できるか試してみる

In [ ]:
text_2 = (
    "Is the following text 'spam'? Answer with 'yes' or 'no':"
    " 'You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award.'"
)

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_2, tokenizer),
    max_new_tokens=23,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

Is the following text 'spam'? Answer with 'yes' or 'no': 'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'

The following text 'spam'? Answer with 'yes' or 'no': 'You are a winner


- 見てわかるように、このモデルは指示に従うのがあまり得意ではない
- これは事前学習のみであり、インストラクションファインチューニングは施されていないためである（インストラクションファインチューニングは次章で扱う）

## 6.5 分類ヘッドの追加

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/lm-head.webp" width=500px>

- このセクションでは、事前学習済みLLMを改変して分類ファインチューニングを行えるようにする
- まずはモデルのアーキテクチャを見直す

In [ ]:
print(model)

- 上図では、第4章で実装したアーキテクチャがわかりやすく整理されて示されている。

- ここでの目的は、出力層を置き換え、それをファインチューニングすることである。

- この目的を達成するために、まずモデル全体を凍結（freeze）し、すべての層を学習不可に設定する。


In [ ]:
for param in model.parameters():
    param.requires_grad = False

- 次に、出力層（`model.out_head`）を置き換える。この層は元々、入力を語彙数（50,257次元）にマッピングする役割を持っていた。

- 今回は「スパム」と「スパムでない」の2クラスを予測する二値分類のためにモデルをファインチューニングするため、出力層を以下のように置き換える。新たな層はデフォルトで学習可能な状態となる。

- 以下のコードをより汎用的に保つために、`BASE_CONFIG["emb_dim"]`（"gpt2-small (124M)" モデルにおいては768）を使用している点に注意せよ。


In [ ]:
torch.manual_seed(123)

num_classes = 2
model.out_head = torch.nn.Linear(in_features=BASE_CONFIG["emb_dim"], out_features=num_classes)

- 技術的には、出力層のみを訓練すれば十分である。

- しかし、[Finetuning Large Language Models](https://magazine.sebastianraschka.com/p/finetuning-large-language-models) で述べたように、実験結果からは追加の層もファインチューニングすることで性能が顕著に向上することが示されている。

- そのため、最後のトランスフォーマーブロックおよび、最後のトランスフォーマーブロックと出力層を接続する `LayerNorm` モジュールも学習可能な状態にする。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/trainable.webp" width=500px>

In [ ]:
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True

- このモデルは、前の章と同様に引き続き使用することができる。

- たとえば、テキスト入力を与えてみよう。


In [ ]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape) # shape: (batch_size, num_tokens)

- 前の章と異なる点は、出力次元が50,257ではなく2次元になっていることである。


In [ ]:
with torch.no_grad():
    outputs = model(inputs)

print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape) # shape: (batch_size, num_tokens, num_classes)

- 前の章で述べたように、各入力トークンに対して1つの出力ベクトルが対応する。

- 今回は4つの入力トークンをモデルに与えたため、上では2次元の出力ベクトルが4つ得られている。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/input-and-output.webp" width=500px>

- 第3章では、各入力トークンが他のすべての入力トークンと接続されるアテンション機構について解説した。

- さらに同章では、GPTのようなモデルで用いられる因果的アテンションマスク（causal attention mask）についても紹介した。この因果マスクにより、各トークンは現在および過去のトークン位置のみにアテンションを向けることができる。

- この因果的アテンション機構に基づき、4番目（最後）のトークンは他のすべてのトークンの情報を含む唯一のトークンであるため、最も多くの情報を保持している。

- したがって、我々は特にこの最後のトークンに注目しており、スパム分類タスクのためにこのトークンをファインチューニングする。


In [ ]:
print("Last output token:", outputs[:, -1, :])

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/attention-mask.webp" width=200px>

## 6.6 分類損失と精度の計算
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-3.webp?1" width=500px>

- 損失の計算方法を説明する前に、モデルの出力がどのようにクラスラベルへと変換されるのかを簡単に確認しておく。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/class-argmax.webp" width=600px>

In [ ]:
print("Last output token:", outputs[:, -1, :])

- 第5章と同様に、出力（ロジット）を `softmax` 関数によって確率スコアに変換し、その後、最も高い確率値のインデックス位置を `argmax` 関数で取得する。


In [ ]:
probas = torch.softmax(outputs[:, -1, :], dim=-1)
label = torch.argmax(probas)
print("Class label:", label.item())

- 第5章で説明したように、ここでは `softmax` 関数は必須ではない点に注意せよ。というのも、出力値が最も大きいものは、確率スコアとしても最も高くなるからである。


In [ ]:
logits = outputs[:, -1, :]
label = torch.argmax(logits)
print("Class label:", label.item())

- この概念を応用することで、分類精度（classification accuracy）を計算することができる。これは、与えられたデータセット内で正しく予測された割合を求める指標である。

- 分類精度を算出するには、前述の `argmax` に基づく予測コードをデータセット内のすべてのサンプルに適用し、正解の予測数の割合を以下のように計算すればよい：


In [ ]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]  # Logits of last output token
            predicted_labels = torch.argmax(logits, dim=-1)

            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break
    return correct_predictions / num_examples

- この関数を用いて、各データセットに対する分類精度を計算してみよう：


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note:
# Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# As of this writing, in PyTorch 2.4, the results obtained via CPU and MPS were identical.
# However, in earlier versions of PyTorch, you may observe different results when using MPS.

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")
#print(f"Running on {device} device.")

model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

torch.manual_seed(123) # For reproducibility due to the shuffling in the training data loader

train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=10)
val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=10)
test_accuracy = calc_accuracy_loader(test_loader, model, device, num_batches=10)

print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

- ご覧のとおり、モデルの予測精度はあまり良くない。これは、まだファインチューニング（訓練）を行っていないためである。

- ファインチューニング（または訓練）を開始する前に、まず訓練中に最適化すべき損失関数を定義する必要がある。

- 目標は、スパム分類におけるモデルの精度を最大化することである。しかしながら、分類精度という指標は微分可能な関数ではない。

- そのため、その代替としてクロスエントロピー損失を最小化する。この損失関数を用いることで、間接的に分類精度を最大化できる（このトピックについて詳しくは、無料で公開されている [Introduction to Deep Learning](https://sebastianraschka.com/blog/2021/dl-course.html#l08-multinomial-logistic-regression--softmax-regression) の講義8を参照のこと）。

- `calc_loss_batch` 関数は第5章で用いたものと同じである。ただし今回は、すべてのトークン `model(input_batch)` を対象にするのではなく、最後のトークン `model(input_batch)[:, -1, :]` のみを最適化対象としている点が異なる。


In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:, -1, :]  # Logits of last output token
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

`calc_loss_loader` 関数は、第5章のものと全く同じである。


In [ ]:
# Same as in chapter 5
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

- `calc_loss_loader` を用いて、訓練開始前に訓練セット・検証セット・テストセットの初期損失を計算する。


In [ ]:
with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)

print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

- 次のセクションでは、損失値を改善し、それに伴って分類精度を向上させるためにモデルを訓練する。


## 6.7 教師ありデータによるモデルのファインチューニング

- このセクションでは、モデルの分類精度を向上させるための訓練関数を定義し、実際に使用する。

- 以下の `train_classifier_simple` 関数は、第5章で事前学習に使用した `train_model_simple` 関数とほぼ同じである。

- 異なる点は次の2つである：
  1. トークン数ではなく、処理した訓練サンプル数（`examples_seen`）を記録している点
  2. 各エポック後にサンプルテキストを出力する代わりに、分類精度を計算している点

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/training-loop.webp?1" width=500px>

In [ ]:
# Overall the same as `train_model_simple` in chapter 5
def train_classifier_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter):
    # Initialize lists to track losses and examples seen
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            examples_seen += input_batch.shape[0] # New: track examples instead of tokens
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Calculate accuracy after each epoch
        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen

- `train_classifier_simple` 関数内で使用されている `evaluate_model` 関数は、第5章で使用したものと同じである。


In [ ]:
# Same as chapter 5
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

- この訓練は、M3 MacBook Air のノートパソコンでは約5分、V100やA100 GPU上では30秒未満で完了する。


In [ ]:
import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

- 第5章と同様に、訓練セットおよび検証セットに対する損失関数の推移を可視化するために、matplotlib を使用する。


In [ ]:
import matplotlib.pyplot as plt

def plot_values(epochs_seen, examples_seen, train_values, val_values, label="loss"):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_values, label=f"Training {label}")
    ax1.plot(epochs_seen, val_values, linestyle="-.", label=f"Validation {label}")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel(label.capitalize())
    ax1.legend()

    # Create a second x-axis for examples seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(examples_seen, train_values, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Examples seen")

    fig.tight_layout()  # Adjust layout to make room
    plt.savefig(f"{label}-plot.pdf")
    plt.show()

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_losses))

plot_values(epochs_tensor, examples_seen_tensor, train_losses, val_losses)

- 上図のように損失が下降傾向にあることから、モデルがうまく学習できていることがわかる。

- また、訓練損失と検証損失が非常に近いことから、モデルが訓練データに過学習していない傾向も確認できる。

- 同様にして、以下に分類精度の推移を可視化することもできる。


In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_accs))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_accs))

plot_values(epochs_tensor, examples_seen_tensor, train_accs, val_accs, label="accuracy")

- 上の精度グラフを見ると、エポック4および5の時点で、モデルが比較的高い訓練および検証精度を達成していることがわかる。

- ただし、訓練関数内で `eval_iter=5` を指定していた点には注意が必要である。これは、訓練セットおよび検証セットの性能をあくまで一部のバッチに基づいて推定していたことを意味する。

- 以下のようにして、訓練・検証・テストセット全体に対する性能を改めて計算することができる。


In [ ]:
train_accuracy = calc_accuracy_loader(train_loader, model, device)
val_accuracy = calc_accuracy_loader(val_loader, model, device)
test_accuracy = calc_accuracy_loader(test_loader, model, device)

print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

- 訓練セットと検証セットにおける性能は、ほぼ同一であることが確認できる。

- しかし、テストセットにおける性能がわずかに低下していることから、モデルが訓練データ、そして学習率などのハイパーパラメータの調整に使用された検証データに対して、わずかに過学習していることがわかる。

- とはいえ、これは通常の範囲内の挙動であり、この性能差は `drop_rate`（ドロップアウト率）やオプティマイザの `weight_decay`（重み減衰）を増やすことで、さらに小さく抑えられる可能性がある。


## 6.8 LLMをスパム分類器として利用する

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-4.webp" width=500px>


- 最後に、ファインチューニングされたGPTモデルを実際に使ってみよう。

- 以下の `classify_review` 関数は、以前に実装した `SpamDataset` と同様のデータ前処理ステップを実装している。

- その後、この関数はモデルから予測された整数クラスラベルを取得し、対応するクラス名を返す。


In [ ]:
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()

    # Prepare inputs to the model
    input_ids = tokenizer.encode(text)
    supported_context_length = model.pos_emb.weight.shape[0]
    # Note: In the book, this was originally written as pos_emb.weight.shape[1] by mistake
    # It didn't break the code but would have caused unnecessary truncation (to 768 instead of 1024)

    # Truncate sequences if they too long
    input_ids = input_ids[:min(max_length, supported_context_length)]

    # Pad sequences to the longest sequence
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0) # add batch dimension

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the classified result
    return "spam" if predicted_label == 1 else "not spam"

- 以下にいくつかの例を使って実際に試してみよう。


In [ ]:
text_1 = (
    "You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award."
)

print(classify_review(
    text_1, model, tokenizer, device, max_length=train_dataset.max_length
))

In [ ]:
text_2 = (
    "Hey, just wanted to check if we're still on"
    " for dinner tonight? Let me know!"
)

print(classify_review(
    text_2, model, tokenizer, device, max_length=train_dataset.max_length
))

- 最後に、後で再訓練せずにモデルを再利用できるように、モデルを保存しておこう。


In [ ]:
torch.save(model.state_dict(), "review_classifier.pth")

- その後、新しいセッションにおいては、以下のようにしてモデルを読み込むことができる。


In [ ]:
model_state_dict = torch.load("review_classifier.pth", map_location=device, weights_only=True)
model.load_state_dict(model_state_dict)

## まとめと重要なポイント

- 分類タスク向けのファインチューニングを行う自己完結型スクリプトは、[./gpt_class_finetune.py](./gpt_class_finetune.py) にある。

- 演習問題の解答は、[./exercise-solutions.ipynb](./exercise-solutions.ipynb) に記載されている。

- さらに興味のある読者は、低ランク適応（LoRA）によるパラメータ効率の高い訓練手法について、[付録E](../../appendix-E) を参照するとよい。
